# Get conversion rates

This notebook only calculates the conversion rates of each Asset traded to BTC and USDT values.

In [ ]:
import sys
sys.path.append("../scripts")
import UTC_time_converter as UTC
from binance.client import Client
import numpy as np
import pickle as pk
import time
import binance.exceptions as Exceptions
from ratelimiter import RateLimiter
import pandas as pd

In [ ]:
#Initialize client
api_key, api_secret = open('../data/client_info.txt').read().split('\n')
client = Client(api_key, api_secret)

In [ ]:
#Ping the server
client.ping()

### Get updated API rate limit
Request weight of `client.get_aggregate_trades()` = 1 per request.

In [ ]:
#Get request weight per minute
API_RATELIMIT = client.get_exchange_info()['rateLimits'][0]['limit']
print(client.get_exchange_info()['rateLimits'][0])
print(API_RATELIMIT)

In [ ]:
#define rate limiter
rate_limiter = RateLimiter(max_calls=0.9*API_RATELIMIT/1, period=60) #period is in units of seconds

#check my math. If it doesn't print True, I can't do math
if 0.9*API_RATELIMIT/1 < API_RATELIMIT : print(True)

In [ ]:
with open('../data/trade_history.pk', 'rb') as f: 
    trade_history = pk.load(f) 
trade_history['Date'] = trade_history.time.apply(lambda x: UTC.milliseconds_to_date(int(x)))

In [ ]:
#Get baseAsset and quoteAsset information
trading_pair_info = pd.DataFrame(client.get_exchange_info()['symbols'])
trading_pair_info[:3]

In [ ]:
trade_history['baseAsset'] = [trading_pair_info.loc[trading_pair_info.symbol == i, 'baseAsset'].values[0] for i in trade_history.symbol]

trade_history['quoteAsset'] = [trading_pair_info.loc[trading_pair_info.symbol ==i, 'quoteAsset'].values[0] for i in trade_history.symbol]


## Define functions to get conversion rates

In [ ]:
def reversible_query(asset1, asset2, startTime, endTime):
    try:
        p = client.get_aggregate_trades(symbol = asset1+asset2,
                                         startTime = startTime,
                                         endTime = endTime)
        if len(p)> 0 : 
            price = float(p[0]['p'])
            return(price)
        if len(p)==0:
            raise AssertionError
    except: 
        #print(symbol, e.status_code, e.message)
        try:
            p = client.get_aggregate_trades(symbol = asset2+asset1,
                                                 startTime = startTime,
                                                 endTime = endTime)
                
            if len(p)> 0 : 
                price = float(p[0]['p'])
                return(price)
            
            if len(p)==0:
                raise AssertionError
                
        except: 
            return(None)
        
def historical_price(baseAsset, quoteAssetNew, startTime, endTime, quoteAsset = False, quickPrice = False):
    
    '''
    Get historical open price for a trading pair
    baseAsset: <str> original baseAsset from Binance trade query
    quoteAssetNew: <str> new quote asset for coin conversion
    quoteAsset: <str> original quoteAsset from Binance trade query
    quickPrice: <float> price of trading pair for given baseAsset and quoteAsset from Binance trade query
    startTime: <float> time in milliseconds
    endTime: <float> time in milliseconds
    
    example: try to find DOGEBTC price
    baseAsset = DOGE, quoteAsset = BNB, quoteAssetNew = BTC
    cannot find DOGEBTC(p1), so use DOGEBNB (p2) and BNBBTC(p3) to calculate DOGEBTC(p1)   
    '''
    

    if quoteAssetNew == quoteAsset and quickPrice:
        return(float(quickPrice))
        
    if quoteAssetNew == baseAsset:
        return(float(1))
    
    if quoteAssetNew != quoteAsset and quoteAssetNew != baseAsset :
        
        p1 = reversible_query(baseAsset,quoteAssetNew, startTime = startTime, endTime = endTime)
        
        if p1: return(p1)
        
        if not quickPrice: 
            p2 = reversible_query(baseAsset,quoteAsset, startTime = startTime, endTime = endTime)
            if not p2: return(np.nan)
            
        p3 = reversible_query(quoteAsset, quoteAssetNew, startTime = startTime, endTime = endTime)
        
        if p3 and quickPrice: 
            price = p3*float(quickPrice)
            return(price)
        
        if p3 and p2: 
            price = p3*p2
            return(price)


### Calculate BTC value of baseAsset and commissionAsset

In [ ]:
t0 = time.time()
with rate_limiter:
    trade_history['baseAssetBTC'] = trade_history.apply(lambda x: historical_price(baseAsset = x.baseAsset,
                                                                                       quoteAssetNew = 'BTC',
                                                                                       quickPrice = x.price,
                                                                                       startTime = x.time,
                                                                                       quoteAsset = x.quoteAsset,
                                                                                       endTime = x.time+50000), axis = 1)
    
    trade_history['commissionAssetBTC'] = trade_history.apply(lambda x: x.baseAssetBTC \
                                                              if x.commissionAsset == x.baseAsset and x.baseAssetBTC \
                                                              else (x.price \
                                                                    if x.commissionAsset == x.baseAsset and x.baseAsset =='BTC' \
                                                                    else historical_price(baseAsset = x.commissionAsset, 
                                                                                          quoteAssetNew = 'BTC', 
                                                                                          startTime = x.time,
                                                                                          quoteAsset = x.baseAsset,
                                                                                          endTime = x.time+50000)) ,\
                                                              axis = 1)
t1 = time.time()
print(t1-t0, 's')

### Deal with NULLs

In [ ]:
#Check for missing values
trade_history.isnull().any(axis=1).shape

In [ ]:
t0 = time.time()
with rate_limiter:
    trade_history['commissionAssetBTC'] = trade_history.apply(lambda x: historical_price(baseAsset = x.commissionAsset, 
                                                                                          quoteAssetNew = 'BTC', 
                                                                                          startTime = x.time,
                                                                                          quoteAsset = x.quoteAsset,
                                                                                          endTime = x.time+50000) \
                                                              if np.isnan(x.commissionAssetBTC) \
                                                              else x.commissionAssetBTC,
                                                              axis = 1)
    trade_history['commissionAssetBTC'] = trade_history.apply(lambda x: historical_price(baseAsset = x.commissionAsset, 
                                                                                          quoteAssetNew = 'BTC', 
                                                                                          startTime = x.time,
                                                                                          quoteAsset = 'USDT',
                                                                                          endTime = x.time+50000) \
                                                              if np.isnan(x.commissionAssetBTC) \
                                                              else x.commissionAssetBTC,
                                                              axis = 1)
t1 = time.time()
print(t1-t0, 's')

In [ ]:
trade_history[:5]

### Calculate USDT value of baseAsset and commissionAsset

In [ ]:
t0 = time.time()
with rate_limiter:
    trade_history['baseAssetUSDT'] = trade_history.apply(lambda x: historical_price(baseAsset = x.baseAsset,
                                                                                       quoteAssetNew = 'USDT',
                                                                                       quickPrice = x.price,
                                                                                       startTime = x.time,
                                                                                       quoteAsset = x.quoteAsset,
                                                                                       endTime = x.time+50000), axis = 1)
    trade_history['commissionAssetUSDT'] = trade_history.apply(lambda x: x.baseAssetUSDT \
                                                                        if x.commissionAsset == x.baseAsset and x.baseAssetUSDT \
                                                                        else
                                                                        historical_price(baseAsset = x.commissionAsset,
                                                                                         quoteAssetNew = 'USDT',
                                                                                         startTime = x.time,
                                                                                         quoteAsset = x.baseAsset,
                                                                                         endTime = x.time+50000), axis = 1)

t1 = time.time()
print(t1-t0, 's')


### Deal with NULLs

In [ ]:
#Check for missing values
trade_history.isnull().any(axis=1).shape

In [ ]:
t0 = time.time()
with rate_limiter:
    trade_history['baseAssetUSDT'] = trade_history.apply(lambda x: historical_price(baseAsset = x.baseAsset,
                                                                                       quoteAssetNew = 'USDT',
                                                                                       quickPrice = x.price,
                                                                                       startTime = x.time,
                                                                                       quoteAsset = x.quoteAsset,
                                                                                       endTime = x.time+50000) \
                                                         if np.isnan(x.baseAssetUSDT) \
                                                         else x.baseAssetUSDT, axis = 1)
    trade_history['commissionAssetUSDT'] = trade_history.apply(lambda x: historical_price(baseAsset = x.commissionAsset,
                                                                                         quoteAssetNew = 'USDT',
                                                                                         startTime = x.time,
                                                                                         quoteAsset = x.baseAsset,
                                                                                         endTime = x.time+50000) \
                                                               if np.isnan(x.commissionAssetUSDT) \
                                                               else x.commissionAssetUSDT, axis = 1)

t1 = time.time()
print(t1-t0, 's')

In [ ]:
#Check for missing values
trade_history.isnull().any(axis=1).shape

In [ ]:
#Convert to dataframe and SAVE
with open('../data/trade_history_converted.pk', 'wb') as f: 
    pk.dump(trade_history, f) 